<a href="https://colab.research.google.com/github/MarcioB1999/ALC/blob/main/AlcAutovalor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install matplotlib-venn
import numpy as np
from numpy import linalg as LA
from scipy.linalg import lu_factor

**Obs: Existem condições a serem atendidas pelas entradas das funções, nas quais não é especificado pelos manuais**

**Método Leverrier Faddeev**

**Manual:** \\
O parâmetro de entrada é uma matriz $\mathbf{A} \in M(n\times n)$, e a saída será um vetor $\mathbf{v}$, com os autovalores, e uma matriz $\mathbf{Q} \in M(n\times n^{2})$ dos autovetores, e por último a matriz inversa $\mathbf{A^{-1}} \in M(n\times n)$.

In [17]:
def LeverrierFaddeev(A):
   n=np.shape(A)[0]
   Def=np.zeros((n,n))
   Q=np.zeros((n,n**2))
   ava=np.zeros(n+1)
   ava[0]=((-1)**(n))*1
   ava[1:n+1], C= aux1(A)
   ava[1:n+1]*=((-1)**(n+1))
   v=np.roots(ava)
   j=np.shape(v)[0]
   Q=np.zeros((n,j*n))
   for i in range(j):
     Q[0:n,i*n:(i+1)*n]=aux2(v[i], C, n)
   if(np.linalg.det(A)!=0):
      Def=(1/ava[n])*C[0:n,n*(n-2):n*(n-1)]
   return v, Q, Def

In [4]:
def aux1(A):
   q=[]
   n=np.shape(A)[0]
   C=np.zeros((n,n**2))
   I=np.eye(n)
   Ak=A
   for i in range(n):
      q.append(np.trace(Ak)/(i+1))
      B=Ak-q[i]*I
      C[0:n,i*n:(i+1)*n]=B[:]
      Ak=A.dot(B)
   return q, C

In [5]:
def aux2(av, C, n):
   Q=(av**(n-1))*np.eye(n)
   for i in range(n-1):
      Q+=(av**(n-i-2))*C[0:n,i*n:(i+1)*n]
   return Q

**Método da Potência**

**Manual:** \\
O parâmetro de entrada é uma matriz $\mathbf{A} \in M(n\times n)$, e a saída seu maior autovalor $\alpha$ em valor absoluto e seu autovetor associado $\mathbf{v} \in \mathbb{R}^{n}$.

In [18]:
def Potencia(A):
   erro=np.zeros(2)
   n=np.shape(A)[0]
   z=np.random.rand(n)
   erro[0]=10**(3)
   apr=np.zeros(n)
   aprant=np.zeros(n)
   while(erro[0]>10**(-2)):
      y=(1/LA.norm(z, np.inf))*z
      z=(np.array(A.dot(y.T))).reshape(-1)
      if(LA.norm(z)<=10**(-2) or LA.norm(y)<=10**(-2)):
        return 0, np.zeros(n)
      apr=(np.array(z)/np.array(y)).reshape(n)
      erro[0]=10**(5)
      for i in range(n):
         aux=abs(apr[i]-aprant[i])
         if(aux<erro[0]):
           erro[0]=aux
           erro[1]=i
      aprant=apr
   return apr[int(erro[1])], y

**Potência Inversa**

**Manual:** \\
O parâmetro de entrada é uma matriz $\mathbf{A} \in M(n\times n)$, e a saída seu menor autovalor $\alpha$ em valor absoluto e seu autovetor associado $\mathbf{v} \in \mathbb{R}^{n}$.

In [20]:
def PotenciaInv(A):
   valor, vetor=questao6(LA.inv(A))
   return 1/valor, vetor

**Método da Deflação com o Método da Potência**

**Manual:** \\
O parâmetro de entrada é uma matriz $\mathbf{A} \in M(n\times n)$, e a saída é um $\mathbf{v} \in \mathbb{R}^{n}$ com seus autovalores, e uma matriz $\mathbf{C} \in M(n\times n)$ com os autovetores associados.

In [21]:
def Deflacao(A):
   n=np.shape(A)[0]
   autvalor=np.zeros(n)
   autvetor=np.zeros((n,n))
   I=np.eye(n)
   for i in range(n):
      while True:
          autvalor[i], autvetor[i,:]=questao6(A)
          if not (autvetor[i,:]==(np.zeros(n))).all():
              break
      u=autvetor[i,:]
      x=u/((LA.norm(u))**(2))
      A=A.dot(I-np.outer(u, x))
   return autvalor, autvetor

**Método Rutishauser**

**Manual:** \\
O parâmetro de entrada é uma matriz $\mathbf{A} \in M(n\times n)$ e um $\alpha$ que será a precisão, e a saída será um vetor $\mathbf{v}$, com os autovalores.

In [9]:
def Rutishauser(A, precisao):
    n=np.shape(A)[0]
    a=np.zeros(n)
    erro=10**(2)
    while(erro>precisao):
         aux, pv=lu_factor(A)
         L, U = np.tril(aux, k=-1) + np.eye(n), np.triu(aux)
         L=A.dot(LA.inv(U))
         A=U.dot(L)
         erro=aux3(A, n)
    for i in range(n):
       a[i]=A[i,i]
    return a

In [10]:
def aux3(A, n):
   erro=10**(3)
   for i in range(1,n):
        auxi=abs(min((np.array(A[i,0:i])).reshape(i)))
        if(auxi<erro):
           erro=auxi
   return erro

In [19]:
 
B=np.matrix([[1, 2, 4], [3, 8, 14], [2, 6, 13]])
Av, vv, In=LeverrierFaddeev(B)
print("autovalores")
print(Av)
#print(vv)
#print(In)
av, y=Potencia(B)
print("maior autovalor")
print(av)
#print(y)
Av, y=PotenciaInv(B)
print("menor autovalor")
print(Av)
#print(y)
Av=questao9(B,10**(-5))
print("autovalores")
print(Av)
K, H=Deflacao(B)
print("autovalores")
print(K)

autovalores
[20.71028893  1.          0.28971107]
maior autovalor
20.709825700363442
menor autovalor
0.2897112553632714
autovalores
[20.7098113   1.00860828  0.28158041]
autovalores
[20.71024705  0.99916001  0.29118872]
